# HTS [Part 3]

In [1]:
%pylab inline
import sys
from ipywidgets import interact

Populating the interactive namespace from numpy and matplotlib


In [2]:
sys.path.append('src')
from motorControl import *
from trajectoryPlanning import *
from util import *

## Load Absolute Home, Upper right corner, out of the way, aligned with sharpie marks
f = open('savedPositions/absolute/home.p', 'r')
home = pickle.load(f)
f.close()

In [3]:
rc = connect(portName = "/dev/tty.usbserial-A9ETDN3N")
rc.Open()

1

### Initialized Motor Classes

- Really important to initialze these first to restore positions from last session!

In [332]:
#Initialize motor objects for each motor:
vL = Motor(address = 0x81, motorNumber = 2, rc = rc, signFlipped = True, \
           motorCounter = 0, kPID = [1e-2, 1.0])
vR = Motor(address = 0x81, motorNumber = 1, rc = rc, signFlipped = True, \
           motorCounter = 1, kPID = [1e-2, 1.0])
LR = Motor(address = 0x80, motorNumber = 1, rc = rc, signFlipped = True, 
           motorCounter = 2, kPID = [1e-2, 1.0])
FB = Motor(address = 0x80, motorNumber = 2, rc = rc, signFlipped = False, \
           motorCounter = 3, kPID = [1e-2, 1.0])
yaw = Motor(address = 0x82, motorNumber = 2, rc = rc, signFlipped = True, \
            motorCounter = 4, kPID = [1e-2, 1.0])
pitch = Motor(address = 0x82, motorNumber = 1, rc = rc, signFlipped = True, \
              motorCounter = 5, kPID = [1e-2, 1.0])

#Keep in a nice motor list:
motors = [vL, vR, LR, FB, yaw, pitch]

In [333]:
getPositions(motors)

[-42303, 42272, 15038, 114355, -431, 10846]

In [334]:
home

[2, 0, 30938, 0, 0, 0]

In [368]:
MC = ManualControl(rc)

interact(MC.manualControl, leftUD = (-50, 50), rightUD = (-50, 50), leftRight = (-127, 127), \
         fB = (-127, 127), tilt = (-50, 50), pan = (-127, 127))

<function ipywidgets.widgets.interaction.<lambda>>

In [365]:
stopAll(rc)
savePositions(motors)
getPositions(motors)

[-2964, -1607, -49498, 97101, -65458, 28317]

In [357]:
levelRig(motors, home, rc, moveTime = 5.0)

Rig is not level!, difference from home = 1351
Rig Leveled! Difference from home = 1359


## Save absolute home

- Only needed for hard resets

In [43]:
#Upper right corner, out of the way, aligned with sharpie marks
# f = open('savedPositions/absolute/home.p', 'wb')
# pickle.dump(getPositions(motors), f, protocol=pickle.HIGHEST_PROTOCOL)
# f.close()

## Save Relative Position

In [369]:
# delta = np.array(getPositions(motors))-np.array(home)
# f = open('savedPositions/relative/HTS/reverseB.p', 'wb')
# pickle.dump(delta, f, protocol=pickle.HIGHEST_PROTOCOL)
# f.close()

## Load Relative Positions

In [375]:
#Position A, directly overhead in center, pointing down
f = open('savedPositions/relative/HTS/A.p', 'r')
A = pickle.load(f)+np.array(home) #Return to absolute coords
f.close()

#Position B, side view, slightly angled down. 
f = open('savedPositions/relative/HTS/B.p', 'r')
B = pickle.load(f)+np.array(home) #Return to absolute coords
f.close()

#Position C, side view, on level
f = open('savedPositions/relative/HTS/C.p', 'r')
C = pickle.load(f)+np.array(home) #Return to absolute coords
f.close()

#Position D, Almost overhead, angled down
f = open('savedPositions/relative/HTS/D.p', 'r')
D = pickle.load(f)+np.array(home) #Return to absolute coords
f.close()

#Position E, Slightly more overhead than D, angled down
f = open('savedPositions/relative/HTS/E.p', 'r')
E = pickle.load(f)+np.array(home) #Return to absolute coords
f.close()

#fastA
f = open('savedPositions/relative/HTS/fastA.p', 'r')
fastA = pickle.load(f)+np.array(home) #Return to absolute coords
f.close()

#fastB
f = open('savedPositions/relative/HTS/fastB.p', 'r')
fastB = pickle.load(f)+np.array(home) #Return to absolute coords
f.close()

#fastC
f = open('savedPositions/relative/HTS/fastC.p', 'r')
fastC = pickle.load(f)+np.array(home) #Return to absolute coords
f.close()

#fastD
f = open('savedPositions/relative/HTS/fastD.p', 'r')
fastD = pickle.load(f)+np.array(home) #Return to absolute coords
f.close()

#reverseA
f = open('savedPositions/relative/HTS/reverseA.p', 'r')
reverseA = pickle.load(f)+np.array(home) #Return to absolute coords
f.close()

#reverseB
f = open('savedPositions/relative/HTS/reverseB.p', 'r')
reverseB = pickle.load(f)+np.array(home) #Return to absolute coords
f.close()



## Move the rig

In [389]:
targetPositions = reverseB
totalTime = 15.0
rampTime = 1.0

startingPositions = getPositions(motors)
lookAheadTime = 1.0
tolerance = 100.0 #Anything less than this many ticks we're calling "Not a move"

motorsToMove = []
targetPositionsToMove = []
for i, motor in enumerate(motors):
    if abs(motor.getPosition()-targetPositions[i]) > tolerance:
        motorsToMove.append(motor)
        targetPositionsToMove.append(targetPositions[i])
        
trajectories = []
for i, motor in enumerate(motorsToMove):
    trajectories.append(SimpleQuadraticTrajectory(tu = rampTime, tt = totalTime, \
                                            p1 = motor.getPosition(), p2 = targetPositionsToMove[i]))
    
print np.array(targetPositions) - startingPositions
print 'Planning to move ' +  str(len(motorsToMove)) + ' motors.'

[    0     0     0 97475   840 15992]
Planning to move 3 motors.


In [390]:
for motor in motorsToMove:
    motor.initialize(targetVelocityMin = -15000.0, targetVelocityMax = 15000.0)
    motor.clearTracking()
    
startTime = time.time()
timeElapsed = 0.0
    
while timeElapsed < totalTime:
    timeElapsed = time.time()-startTime
    
    for i, motor in enumerate(motorsToMove):
        lookAheadValue = trajectories[i].compute(timeElapsed + lookAheadTime)
        motor.controlledMove(targetPosition = lookAheadValue, timeToReach = lookAheadTime)

stopAll(rc)
savePositions(motors)
print getPositions(motors), targetPositions
print getPositions(motors) - np.array(targetPositions)

excess voltage on motor 3
excess voltage on motor 3
excess voltage on motor 3
excess voltage on motor 3
excess voltage on motor 3
excess voltage on motor 3
excess voltage on motor 3
excess voltage on motor 3
excess voltage on motor 3
excess voltage on motor 3
excess voltage on motor 3
excess voltage on motor 3
excess voltage on motor 3
excess voltage on motor 3
excess voltage on motor 3
excess voltage on motor 3
excess voltage on motor 3
excess voltage on motor 3
excess voltage on motor 3
excess voltage on motor 3
excess voltage on motor 3
excess voltage on motor 3
excess voltage on motor 3
excess voltage on motor 3
excess voltage on motor 3
excess voltage on motor 3
excess voltage on motor 3
excess voltage on motor 3
excess voltage on motor 3
excess voltage on motor 3
excess voltage on motor 3
excess voltage on motor 3
excess voltage on motor 3
excess voltage on motor 3
excess voltage on motor 3
excess voltage on motor 3
excess voltage on motor 3
excess voltage on motor 3
excess volta

In [160]:
stopAll(rc)
savePositions(motors)